# Getting started with TensorFlow's `Dataset` API

In this notebook we use TensorFlow's [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data/) API  to build simple input pipelines from Numpy and TensorFlow arrays existing in memory. This is intended to be used only with very small datasets as it can be considerably inefficient.

More info can be found on the session [Importing Data](https://www.tensorflow.org/guide/datasets) on TensorFlow's page.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.version.VERSION

'2.3.0'

In [3]:
# Create fake data
nsamples = 10
nfeatures = 4
x_numpy = np.random.random((nsamples, nfeatures))
y_numpy = x_numpy.sum(axis=1).astype(int)

In [4]:
# Creating a `Dataset` object
dataset = tf.data.Dataset.from_tensor_slices((x_numpy, y_numpy))
dataset = dataset.shuffle(10)
dataset = dataset.batch(1)
dataset = dataset.repeat(1)

# * Dataset.repeat() concatenates the datataset without signaling the end of one epoch
#   and the beginning of the next one.

In [5]:
for x, y in dataset:
    print(f'x: {x}    y: {y}')

x: [[0.35586413 0.5696162  0.04174093 0.62823897]]    y: [1]
x: [[0.4634437  0.88714993 0.92268097 0.87789459]]    y: [3]
x: [[0.65869466 0.66703969 0.49737143 0.19499465]]    y: [2]
x: [[0.3101127  0.17971135 0.91034815 0.32592887]]    y: [1]
x: [[0.73344892 0.89501475 0.69507097 0.82132055]]    y: [3]
x: [[0.50874734 0.55939823 0.9406282  0.73281027]]    y: [2]
x: [[0.02566974 0.98566664 0.56974207 0.40252372]]    y: [1]
x: [[0.09437051 0.71640226 0.80308381 0.51236862]]    y: [2]
x: [[0.50359572 0.44534517 0.36096608 0.97405061]]    y: [2]
x: [[0.04984346 0.47957477 0.87984861 0.0256002 ]]    y: [1]


In [6]:
# iterate up to the 5th sample
for x, y in dataset.take(5):
    print(f'x: {x}    y: {y}')

x: [[0.50874734 0.55939823 0.9406282  0.73281027]]    y: [2]
x: [[0.4634437  0.88714993 0.92268097 0.87789459]]    y: [3]
x: [[0.02566974 0.98566664 0.56974207 0.40252372]]    y: [1]
x: [[0.35586413 0.5696162  0.04174093 0.62823897]]    y: [1]
x: [[0.73344892 0.89501475 0.69507097 0.82132055]]    y: [3]


### The `Dataset` object can be created also from TensorFlow tensor objects

In [7]:
x_tensor = tf.random.uniform([nsamples, nfeatures])
y_tensor = tf.cast(tf.reduce_sum(x_tensor, axis=1), tf.int32)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))
dataset = dataset.shuffle(10)
dataset = dataset.batch(1)
dataset = dataset.repeat(1)

In [9]:
for x, y in dataset.take(5):
    print(f'x: {x}    y: {y}')

x: [[0.3642919  0.73825693 0.7308357  0.5296619 ]]    y: [2]
x: [[0.0539037  0.30332398 0.08642542 0.9834143 ]]    y: [1]
x: [[0.41189802 0.6670884  0.09735096 0.00721169]]    y: [1]
x: [[0.5984529  0.30378234 0.6065934  0.04713142]]    y: [1]
x: [[0.23360538 0.9726051  0.427588   0.7342621 ]]    y: [2]


## Adding data transformations to the pipeline

Let's say that for our problem it is beneficial to center the features between -0.5 and 0.5. Also, we would like to transform the labels from integers to one-hot encoded. This is can be done with `Dataset`'s method `map`.

In [10]:
# The following transformations are quite simple and can be done
# on a single function, but we will use two different functions
# to show how operations can be put together as a pipeline.

def center(features, label):
    features = features - 0.5
    return features, label

def make_on_hot_labels(features, label):
    return features, tf.one_hot(label, 4)

# simpler with `dataset = dataset.filter(lambda f, l: tf.equal(l, 1))`
def filter_labels(features, label):
    return tf.equal(label, 1)

# simpler with `dataset = dataset.filter(lambda f, l: tf.greater(f[0], 0)`
def filter_features(features, label):
    return tf.greater(features[0], 0)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((x_numpy, y_numpy))
dataset = dataset.filter(filter_labels)
dataset = dataset.map(center)
dataset = dataset.filter(filter_features)
dataset = dataset.map(make_on_hot_labels)
dataset = dataset.shuffle(150)
dataset = dataset.batch(1)
dataset = dataset.repeat(1)

In [12]:
for x, y in dataset.take(5):
    print(f'x: {x}    y: {y}')

## Giving names to `Dataset` components

In [13]:
dataset = tf.data.Dataset.from_tensor_slices({'features': x_numpy, 'label': y_numpy})

In [14]:
def center(sample):
    features = sample['features'] - 0.5
    label = sample['label']
    return {'features': features, 'label': label}

In [15]:
dataset = tf.data.Dataset.from_tensor_slices({'features': x_numpy, 'label': y_numpy})
dataset = dataset.map(center)
dataset = dataset.shuffle(10)
dataset = dataset.batch(1)
dataset = dataset.repeat(1)

In [16]:
for d in dataset.take(5):
    print(f"x: {d['features']}    y: {d['label']}")

x: [[-0.0365563   0.38714993  0.42268097  0.37789459]]    y: [3]
x: [[0.00874734 0.05939823 0.4406282  0.23281027]]    y: [2]
x: [[-0.1898873  -0.32028865  0.41034815 -0.17407113]]    y: [1]
x: [[-0.47433026  0.48566664  0.06974207 -0.09747628]]    y: [1]
x: [[-0.45015654 -0.02042523  0.37984861 -0.4743998 ]]    y: [1]
